[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2024/blob/main/01--edi/07--xml-html.ipynb)

## Złożone formaty

* xml
* html

### XML

    * transfer danych / JPK / Dokumenty / Formularze
    * zagnieżdzone struktury
    * walidacja dokumentu
    * Ogólnodostępne parsery w językach programowania
    * Zorientowany biznesowo

```xml
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
```

### Html


```html
<li class="link">
    <div class="article" data-id="4789115">
        <a href="#4789115">
            <span>85</span>
            <span class="button button submit">wykop</span>
        </a>
    </div>

    <div class="media-content m-reset-float ">
        <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/">
            <img src="https://www.wykop.pl/cdn/c3397993/link_MwUxfsmJdpfvuiy0ICl1l0Yw35VeWgBc,w207h139.jpg" alt="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">
        </a>
    </div>
    <div class="lcontrast m-reset-margin">
        <h2>
            <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                rel="nofollow noopener noreferrer" title="W jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.">W
                jaki sposób Facebook stał się lewicową sektą? Opowieść pracownika firmy.</a>
        </h2>


        <div class="description">
            <p class="text">
                <a href="https://www.wykop.pl/link/4789115/w-jaki-sposob-facebook-stal-sie-lewicowa-sekta-opowiesc-pracownika-firmy/"
                    title="">
                    Tragiczna relacja pracownika, który przez lata obserwował jak serwis promujący wolność słowa
                    zamienił się w ideologiczny rynsztok prześladujący pracowników o zróżnicowanym światopoglądzie.
                </a>
            </p>
        </div>
    </div>
</li>
```

In [1]:
!pip install bs4

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.10.0-py3-none-any.whl (97 kB)
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4 ... done


In [2]:
sample_xml = '''
<article>
    <info>
        <title>Some title</title>
        <authors>
            <author>
                <firstname>Jakub</firstname>
                <lastname>Kanclerz</lastname>
            </author>
        </authors>
    </info>
    <content>
        <section type="header">
            <paragraph>
                some title
            </paragraph>
        </section>
        <section type="main">
            <paragraph>
                some content
            </paragraph>
            <image src="image/src.jpeg"/>
        </section>
    </content>
</article>
'''

In [3]:
import requests
from bs4 import BeautifulSoup as bs
from itertools import chain

In [4]:
loaded = bs(sample_xml)

In [5]:
soups = loaded.select('author firstname')

In [6]:
print(soups[0].text)

Jakub


### Webscrapping

### Lista sklepów biedronka
[https://www.biedronka.pl/pl/sklepy/lista](https://www.biedronka.pl/pl/sklepy/lista)

In [172]:
import requests
import itertools
from collections import namedtuple

In [173]:
URL_TEMPLATE = "https://www.biedronka.pl/pl/sklepy/lista,page,{}"
URL_TEMPLATE = "https://www.biedronka.pl/pl/sklepy/lista,city,krakow,page,{}"
PAGES_TO_SCAN = 10
#PAGES_TO_SCAN = 160

In [186]:
urls = [URL_TEMPLATE.format(p) for p in range(1, PAGES_TO_SCAN)]

In [187]:
responses = (requests.get(url) for url in urls)

In [188]:
contents = (r.text for r in responses)

In [189]:
soups = (bs(html_content) for html_content in contents)

In [190]:
Shop = namedtuple('Shop', ['city', 'zip', 'street'])

In [191]:
example_shop_element = '''
<h4 style="margin-bottom: 10px;">
    Rzeszów            	<span class="shopFullAddress">
        <span class="shopAddress">
            35-078<br>
            Hetmańska 56                    </span>
    </span>
</h4>
'''

In [192]:
def extract_shop(soup):
    try:
        # Extract city name
        city = (
            soup.select_one('div.header h4 a').text.strip()
        )
    except Exception:
        city = None

    try:
        # Extract address
        address = (
            soup.select_one('p.shopAddress').text.strip()
        )
    except Exception:
        address = None

    try:
        # Extract ZIP code
        zip_code = (
            soup.select_one('p.zipCode').text.strip()
        )
    except Exception:
        zip_code = None

    return Shop(city, zip_code, address)

In [193]:
assert extract_shop(bs(example_shop_element)) == Shop('Kraków', '30-211', 'Piastowska 49')

In [194]:
extract_shop(bs(example_shop_element))

Shop(city='Rzeszów', zip='35-078', street='Hetmańska 56')

In [195]:
shop_soups_batch = (soup.select('ul.shopList li.shopListElement ') for soup in soups)

In [196]:
shop_soups = itertools.chain.from_iterable(shop_soups_batch)

In [197]:
shops = (extract_shop(soup) for soup in shop_soups)

In [198]:
shop_as_list = list(shops)

In [199]:
shop_as_list[:10]

[Shop(city='Rzeszów', zip='35-078', street='Hetmańska 56'),
 Shop(city='Kraśnik', zip='23-200', street='Urzędowska 472'),
 Shop(city='Medyka', zip='37-732', street='Przejście graniczne 405'),
 Shop(city='Police', zip='72-011', street='Głowackiego 10'),
 Shop(city='Szczecin', zip='70-781', street='Pomarańczowa 9'),
 Shop(city='Szczecin', zip='70-747', street='Karpia 1'),
 Shop(city='Myślibórz', zip='74-300', street='Lipowa 7'),
 Shop(city='Szczecin', zip='71-123', street='Stanisława Ignacego Witkiewicza 41'),
 Shop(city='Gorzów Wielkopolski', zip='66-400', street='Matejki 46'),
 Shop(city='Otwock', zip='05-400', street='Kołłątaja 61')]

## Utrwalenie rezultatów

In [200]:
import csv
with open('var/biedronkas.csv', 'w') as destination_file:
    writer = csv.writer(destination_file, delimiter=';', quotechar='"')
    for row in shop_as_list:
        writer.writerow(row)

In [204]:
!cat var/biedronkas.csv | head -n 5

Rzeszów;35-078;Hetmańska 56

Kraśnik;23-200;Urzędowska 472

Medyka;37-732;Przejście graniczne 405

Police;72-011;Głowackiego 10

Szczecin;70-781;Pomarańczowa 9



In [205]:
import json
with open('var/biedronkas.data', 'w') as destination_file:
    for row in shop_as_list:
        destination_file.write(json.dumps(row._asdict()) + "\n")

In [206]:
!cat var/biedronkas.data | head -n 5

{"city": "Rzesz\u00f3w", "zip": "35-078", "street": "Hetma\u0144ska 56"}
{"city": "Kra\u015bnik", "zip": "23-200", "street": "Urz\u0119dowska 472"}
{"city": "Medyka", "zip": "37-732", "street": "Przej\u015bcie graniczne 405"}
{"city": "Police", "zip": "72-011", "street": "G\u0142owackiego 10"}
{"city": "Szczecin", "zip": "70-781", "street": "Pomara\u0144czowa 9"}


## Czy posiadając adres potrafisz ustali geolokalizację konkretnego sklepu?

https://letmegooglethat.com/?q=openstreet+map+geocoding

https://www.geoapify.com/openstreetmap-geocoding
https://apidocs.geoapify.com/playground/geocoding/

lub ``Don't Reinvent The Wheel`` ``* Unless You Plan on Learning More About Wheels`
https://geocoder.readthedocs.io/providers/OpenStreetMap.html

### Zadanie, 
Wykorzystaj zbiór danych o adresach sieci sklepów biedronka 

In [217]:
import pickle

In [220]:
with open('var/biedronkas.pickle', 'w') as f:
    pickle.dump(shop_as_list, f)

TypeError: write() argument must be str, not bytes

In [221]:
with open('var/biedronkas.pickle', 'wb') as f:
    pickle.dump(shop_as_list, f)

In [223]:
!cat var/biedronkas.pickle | head

���       ]�(�__main__��Shop����Rzeszów��35-078��
Hetmańska 56�����h�Kraśnik��23-200��Urzędowska 472�����h�Medyka��37-732��Przejście graniczne 405�����h�Police��72-011��Głowackiego 10�����h�Szczecin��70-781��Pomarańczowa 9�����h�Szczecin��70-747��Karpia 1�����h�
Myślibórz��74-300��Lipowa 7�����h�Szczecin��71-123��#Stanisława Ignacego Witkiewicza 41�����h�Gorzów Wielkopolski��66-400��
Matejki 46�����h�Otwock��05-400��Kołłątaja 61�����h�Szczecin��71-610��ul. Stanisława Dubois 5�����h�Gorzów Wielkopolski��66-400��Popławskiego 72�����h�	Pruszków��05-800��Aleje Jerozolimskie 454�����h�Szczecin��71-684��
Obotrycka 6�����h�Szczecin��71-017��
Krakowska 67�����h�Mierzyn��72-006��	Welecka 2�����h�Szczecin��70-841��ul. Goleniowska 87�����h�
Józefosław��05-509��Julianowska 57A�����h�Szczecin��70-473��Wojska Polskiego 31�����h�Szczecin��71-270��Klemensa Janickiego 24�����h�Szczecin��71-073��al. Piastów 22/U1�����h�Sz